<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/kmeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import random
import copy
import datetime
import seaborn as sns

In [2]:
df = pd.read_csv('result_rog_csc_wk_sales.csv')
df.columns = [col.lower() for col in df.columns]
orig_df = df.copy()

In [3]:
df.head()

,week_start_date,week_end_date,consumer_selling_cd,rog_id,total_gross_amt,total_item_qty,total_mkdn_qty,total_mkdn_amt,total_net_amt,total_mkdn_wod_alloc_amt,total_mkdn_pod_alloc_amt
0,2019-12-29,2020-01-04,1024176,AJWL,406.52,314,286,-83.57,322.95,0.0,-0.5035
1,2019-12-29,2020-01-04,858404,AJWL,234.82,118,96,-47.04,187.78,0.0,0.0000
2,2019-12-29,2020-01-04,969941,AJWL,159.20,80,0,0.00,159.20,0.0,0.0000
3,2019-12-29,2020-01-04,861171,AJWL,784.67,563,2,-2.00,782.67,0.0,-0.0461
4,2019-12-29,2020-01-04,857583,AJWL,1713.69,311,294,-177.05,1536.64,0.0,0.0000


In [4]:
df = df[(df.total_item_qty > 0) & (df.total_net_amt > 0)]

In [5]:
#df['total_gross_amt_log'] = np.log(df['total_gross_amt'])
df['total_item_qty_log'] = np.log(df['total_item_qty'])
#df['total_mkdn_qty_log'] = np.log(df['total_mkdn_qty'])
#df['total_mkdn_amt_log'] = np.log(df['total_mkdn_amt'])
df['total_net_amt_log'] = np.log(df['total_net_amt'] + 1)
#df['total_mkdn_wod_alloc_amt_log'] = np.log(df['total_mkdn_wod_alloc_amt'])
#df['total_mkdn_pod_alloc_amt_log'] = np.log(df['total_mkdn_pod_alloc_amt']) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [6]:
df.head()

,week_start_date,week_end_date,consumer_selling_cd,rog_id,total_gross_amt,total_item_qty,total_mkdn_qty,total_mkdn_amt,total_net_amt,total_mkdn_wod_alloc_amt,total_mkdn_pod_alloc_amt,total_item_qty_log,total_net_amt_log
0,2019-12-29,2020-01-04,1024176,AJWL,406.52,314,286,-83.57,322.95,0.0,-0.5035,5.749393,5.780589
1,2019-12-29,2020-01-04,858404,AJWL,234.82,118,96,-47.04,187.78,0.0,0.0000,4.770685,5.240582
2,2019-12-29,2020-01-04,969941,AJWL,159.20,80,0,0.00,159.20,0.0,0.0000,4.382027,5.076423
3,2019-12-29,2020-01-04,861171,AJWL,784.67,563,2,-2.00,782.67,0.0,-0.0461,6.333280,6.663988
4,2019-12-29,2020-01-04,857583,AJWL,1713.69,311,294,-177.05,1536.64,0.0,0.0000,5.739793,7.338004


In [7]:
df_consumer_selling_cd = df['consumer_selling_cd']
df.drop(columns = ['consumer_selling_cd','week_start_date', 'rog_id', 'total_gross_amt', 'total_item_qty', 'total_mkdn_qty', 'total_mkdn_amt', 'total_net_amt', 'total_mkdn_wod_alloc_amt', 'total_mkdn_pod_alloc_amt'], axis = 1 , inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df.head()

,week_end_date,total_item_qty_log,total_net_amt_log
0,2020-01-04,5.749393,5.780589
1,2020-01-04,4.770685,5.240582
2,2020-01-04,4.382027,5.076423
3,2020-01-04,6.333280,6.663988
4,2020-01-04,5.739793,7.338004


In [9]:
df[df.total_net_amt_log == 0]

,week_end_date,total_item_qty_log,total_net_amt_log


In [10]:
df = df.fillna(0)
df.set_index('week_end_date', inplace = True)
df.head()

,total_item_qty_log,total_net_amt_log
week_end_date,,
2020-01-04,5.749393,5.780589
2020-01-04,4.770685,5.240582
2020-01-04,4.382027,5.076423
2020-01-04,6.333280,6.663988
2020-01-04,5.739793,7.338004


In [11]:
df.isna().sum()

total_item_qty_log    0
total_net_amt_log     0
dtype: int64

In [12]:
df.describe()

,total_item_qty_log,total_net_amt_log
count,77064.000000,77064.000000
mean,6.430971,7.142307
std,1.575135,1.432680
min,0.000000,0.173953
25%,5.752573,6.507964
50%,6.716595,7.514358
75%,7.464510,8.079593
max,10.428630,10.138168


In [13]:
X = np.array(df.values)
X

array([[5.74939299, 5.78058918],
       [4.77068462, 5.24058232],
       [4.38202663, 5.07642303],
       ...,
       [4.99043259, 6.59171488],
       [5.29330482, 7.16430266],
       [6.54534966, 6.23539064]])

In [15]:
n_clusters = 5
random_state = 10  # makes sure you get the same results each time
X = np.array(df.values)


def fit_kmeans(X, n_clusters, random_state):
  ## YOUR CODE HERE ## 
  model = KMeans(n_clusters = n_clusters,
           #init = 'random',
           init='k-means++',
           n_init = 10,
           random_state = random_state)

  return model

model = fit_kmeans(X, n_clusters, random_state)

## get cluster prediction and add to DataFrame
# predict k-means classes
df['cluster'] = model.fit_predict(X)
#print(model.cluster_centers_)
print(df)

               total_item_qty_log  total_net_amt_log  cluster
week_end_date                                                
2020-01-04               5.749393           5.780589        2
2020-01-04               4.770685           5.240582        0
2020-01-04               4.382027           5.076423        0
2020-01-04               6.333280           6.663988        2
2020-01-04               5.739793           7.338004        2
...                           ...                ...      ...
2022-10-15               6.527958           7.103330        1
2022-10-15               6.186209           7.888162        1
2022-10-15               4.990433           6.591715        2
2022-10-15               5.293305           7.164303        2
2022-10-15               6.545350           6.235391        2

[77064 rows x 3 columns]


In [ ]:
max_k=20 # max no. of clusters to be evaluated
wcss = []
for i in range(1, max_k):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = random_state)
    kmeans.fit(X)
    # inertia method returns wcss for that model
    wcss.append(kmeans.inertia_)
#plotting the data 
plt.figure(figsize=(5,3))
sns.lineplot(range(1, max_k), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()